In [34]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [35]:
#导入数据

lib_feature=pd.read_csv('lib_input/library_train.csv')
lib_feature.head()

,id,lib_dor_num,lib_date,date,year,month,hour,weekday,lib_hour_07,lib_hour_08,...,lib_month_07,lib_month_08,lib_month_09,lib_month_10,lib_month_11,lib_month_12,lib_spring,lib_summer,lib_autumn,lib_winter
0,3684,5,2013-09-01 08:42:50,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0
1,14314,6,2013-09-01 08:42:59,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0
2,5108,9,2013-09-01 08:43:32,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0
3,5826,7,2013-09-01 08:44:06,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0
4,7730,5,2013-09-01 08:44:25,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0


In [43]:
#获得每个id去图书馆的总天数

lib_day=DataFrame(lib_feature.groupby(['id','date'])['id'].count())
lib_day.to_csv('lib_input/lib_day.csv',index=True)
lib_day=pd.read_csv('lib_input/lib_day.csv')
lib_day_count=DataFrame(lib_day.groupby('id')['id.1'].count())
lib_day_count.columns=['lib_day_sum']

In [45]:
#剔除无用列

lib_feature.drop(['lib_dor_num','lib_date','date','year','hour','month','weekday'],axis=1,inplace=True)

In [46]:
#根据id聚合求解各个特征的产生的总数

count_columns=list(lib_feature.columns)[1:]

#count_columns
count_pivot=lib_feature.pivot_table(index='id',values=count_columns,aggfunc=np.count_nonzero)

In [47]:
#连接表

lib_feature_f=count_pivot.join(DataFrame(lib_day_count))

In [50]:
#根据id聚合求解各个特征均值(各特征去图书馆次数占总数比例)

a=['id']
mean_columns=[x for x in list(lib_feature.columns) if x not in a]
mean_pivot=lib_feature.pivot_table(index='id',values=mean_columns,aggfunc=np.mean)

#对列名重命名
mean_pivot.columns=[x+'_mean' for x in list(mean_pivot.columns)]

#连接两表
lib_feature_f=lib_feature_f.join(mean_pivot)


In [54]:
#定义id关于每一列去图书馆的均值(每天)

def get_colunms_mean(df_sum,df_count):
    a=list(df_count.columns)
    columns_mean=DataFrame(df_count[a[0]])#初始化表格大小
    columns_mean.columns=['id_count']
    for x in range(len(a)):
        columns_count_x=df_count[a[x]].values
        columns_sum=df_sum['lib_day_sum'].values
        columns_mean[a[x]+'_per_day']=np.divide.reduce([columns_count_x,columns_sum],axis=0)
        x+=1
    columns_mean.drop('id_count',axis=1,inplace=True)
    return columns_mean

#运用定义的函数
lib_columns_mean=get_colunms_mean(lib_feature_f,count_pivot)

#连接总表
lib_feature_f=lib_feature_f.join(lib_columns_mean)

In [61]:
#每天去图书馆的平均次数

lib_feature_f['lib_count_sum']=lib_feature.groupby('id')['id'].count()
lib_feature_f['lib_per_day']=lib_feature_f['lib_count_sum']/lib_feature_f['lib_day_sum']

In [63]:
lib_feature_f.to_csv('lib_input/lib_train_feature_f.csv',index='id')

In [48]:
count_pivot.head()

,lib_afternoon,lib_autumn,lib_evening,lib_hour_07,lib_hour_08,lib_hour_09,lib_hour_10,lib_hour_11,lib_hour_12,lib_hour_13,...,lib_month_09,lib_month_10,lib_month_11,lib_month_12,lib_morning,lib_resting_day,lib_spring,lib_summer,lib_winter,lib_working_day
id,,,,,,,,,,,,,,,,,,,,,
0,10,11,6,0,0,0,0,0,0,0,...,0,6,5,2,0,1,1,0,4,15
1,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2,0,2
10,1,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,3,4
19,17,0,2,0,0,0,0,0,0,1,...,0,0,0,0,0,2,15,4,0,17
20,1,9,12,0,0,0,1,0,0,1,...,1,7,1,2,1,1,1,2,2,13


In [25]:
lib_feature.head()

,id,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,...,lib_month_07,lib_month_08,lib_month_09,lib_month_10,lib_month_11,lib_month_12,lib_spring,lib_summer,lib_autumn,lib_winter
0,3684,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,14314,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,5108,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,5826,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,7730,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [62]:
lib_feature_f.head()

,lib_afternoon,lib_autumn,lib_evening,lib_hour_07,lib_hour_08,lib_hour_09,lib_hour_10,lib_hour_11,lib_hour_12,lib_hour_13,...,lib_month_11_per_day,lib_month_12_per_day,lib_morning_per_day,lib_resting_day_per_day,lib_spring_per_day,lib_summer_per_day,lib_winter_per_day,lib_working_day_per_day,lib_count_sum,lib_per_day
id,,,,,,,,,,,,,,,,,,,,,
0,10,11,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,16,1.230769
1,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,2,0,2,2,2.000000
10,1,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,4,1.333333
19,17,0,2,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,19,1.900000
20,1,9,12,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,14,1.076923
